In [1]:
from pathlib import Path
import pandas as pd

# Use repo-relative `pr` directory (Windows)
print("cwd:", Path.cwd())
base_path = Path("pr")
print("resolved:", base_path.resolve())
print("exists:", base_path.exists())

records = []
if base_path.exists():
    for f in sorted(base_path.rglob("*")):
        if not f.is_file():
            continue

        read_ok = True
        detail = "ok"
        n_rows = None
        n_cols = None

        try:
            ext = f.suffix.lower()

            if ext == ".csv":
                df_tmp = pd.read_csv(f)
                n_rows, n_cols = df_tmp.shape
            elif ext in [".tsv", ".txt"]:
                df_tmp = pd.read_csv(f, sep="\t")
                n_rows, n_cols = df_tmp.shape
            elif ext == ".json":
                df_tmp = pd.read_json(f)
                n_rows, n_cols = df_tmp.shape
            elif ext == ".parquet":
                df_tmp = pd.read_parquet(f)
                n_rows, n_cols = df_tmp.shape
            elif ext in [".xlsx", ".xls"]:
                df_tmp = pd.read_excel(f)
                n_rows, n_cols = df_tmp.shape
            else:
                # Validación básica de lectura para otros tipos
                with open(f, "rb") as fh:
                    _ = fh.read(1024)

        except Exception as e:
            read_ok = False
            detail = str(e)

        records.append(
            {
                "file_name": f.name,
                "file_path": str(f),
                "size_bytes": f.stat().st_size,
                "read_ok": read_ok,
                "rows": n_rows,
                "cols": n_cols,
                "detail": detail,
            }
        )
else:
    print(f"No existe la ruta: {base_path}")

ds = pd.DataFrame(records)
display(ds)
print(f"Total de archivos encontrados: {len(ds)}")

cwd: c:\Users\pgome\Documents\GitHub\TESIS
resolved: C:\Users\pgome\Documents\GitHub\TESIS\pr
exists: True


,file_name,file_path,size_bytes,read_ok,rows,cols,detail
0,.DS_Store,pr\.DS_Store,6148,True,None,None,ok
1,CR2MET_pr_v2.5_day_1966_01_005deg.nc,pr\CR2MET_pr_v2.5_day_1966_01_005deg.nc,22010448,True,None,None,ok
2,CR2MET_pr_v2.5_day_1966_02_005deg.nc,pr\CR2MET_pr_v2.5_day_1966_02_005deg.nc,19898424,True,None,None,ok
3,CR2MET_pr_v2.5_day_1966_03_005deg.nc,pr\CR2MET_pr_v2.5_day_1966_03_005deg.nc,22010448,True,None,None,ok
4,CR2MET_pr_v2.5_day_1966_04_005deg.nc,pr\CR2MET_pr_v2.5_day_1966_04_005deg.nc,21306440,True,None,None,ok
...,...,...,...,...,...,...,...
668,CR2MET_pr_v2.5_day_2021_08_005deg.nc,pr\CR2MET_pr_v2.5_day_2021_08_005deg.nc,22010448,True,None,None,ok
669,CR2MET_pr_v2.5_day_2021_09_005deg.nc,pr\CR2MET_pr_v2.5_day_2021_09_005deg.nc,21306440,True,None,None,ok
670,CR2MET_pr_v2.5_day_2021_10_005deg.nc,pr\CR2MET_pr_v2.5_day_2021_10_005deg.nc,22010448,True,None,None,ok
671,CR2MET_pr_v2.5_day_2021_11_005deg.nc,pr\CR2MET_pr_v2.5_day_2021_11_005deg.nc,21306440,True,None,None,ok


Total de archivos encontrados: 673


In [2]:
# Cargar dataset CR2MET desde los archivos en `pr` y definir punto de Stgo centro
import xarray as xr
from pathlib import Path

pr_dir = Path('pr')
files = sorted(pr_dir.glob('CR2MET_pr_v2.5_day_*.nc'))
print('Archivos encontrados:', len(files))
if not files:
    raise RuntimeError('No se encontraron archivos CR2MET en pr/')

# Abrir con xarray (usar dask para eficiencia si está disponible)
ds = xr.open_mfdataset([str(p) for p in files], combine='by_coords', parallel=True, chunks={'time': 365})
print(ds)

# Coordenadas de Santiago centro
lat_sc = -33.45
lon_sc = -70.66
print('lat_sc =', lat_sc, 'lon_sc =', lon_sc)

pr_sc = ds["pr"].sel(lat=lat_sc, lon=lon_sc, method="nearest")

# Intentar seleccionar el punto más cercano usando nombres de coordenadas comunes
lat_name = next((n for n in ds.coords if 'lat' in n.lower()), None)
lon_name = next((n for n in ds.coords if 'lon' in n.lower()), None)
print('lat coord:', lat_name, 'lon coord:', lon_name)
if lat_name and lon_name:
    point = ds.sel({lat_name: lat_sc, lon_name: lon_sc}, method='nearest')
    print('Coordenadas del punto más cercano:', float(point[lat_name]), float(point[lon_name]))
else:
    print('No se encontraron coordenadas lat/lon en el dataset. Coordenadas disponibles:', list(ds.coords))

Archivos encontrados: 672


C:\Users\pgome\AppData\Local\Temp\ipykernel_12752\412657175.py:12: FutureWarning: In a future version of xarray the default value for data_vars will change from data_vars='all' to data_vars=None. This is likely to lead to different results when multiple datasets have matching variables with overlapping values. To opt in to new defaults and get rid of these warnings now use `set_options(use_new_combine_kwarg_defaults=True) or set data_vars explicitly.
  ds = xr.open_mfdataset([str(p) for p in files], combine='by_coords', parallel=True, chunks={'time': 365})


<xarray.Dataset> Size: 72GB
Dimensions:  (time: 20454, lat: 800, lon: 220)
Coordinates:
  * time     (time) datetime64[ns] 164kB 1966-01-01 1966-01-02 ... 2021-12-31
  * lat      (lat) float64 6kB -56.98 -56.93 -56.88 ... -17.12 -17.07 -17.02
  * lon      (lon) float64 2kB -76.97 -76.92 -76.88 ... -66.12 -66.08 -66.03
Data variables:
    pr       (time, lat, lon) float64 29GB dask.array<chunksize=(31, 800, 220), meta=np.ndarray>
    pr_sd    (time, lat, lon) float64 29GB dask.array<chunksize=(31, 800, 220), meta=np.ndarray>
    cl_mask  (time, lat, lon) float32 14GB dask.array<chunksize=(31, 800, 220), meta=np.ndarray>
Attributes: (12/28)
    creation_date:        29-Aug-2022 18:44:17
    authors:              Boisier et al.
    obs_file:             MERGE5_QC_UCC_pr_day_1979_2021.csv
    topo_res:             005 025 05
    ls_vars_all:          pr logpr prc prmax pr2d iqu iqv tcw cape
    ls_vars_o15:          iqu iqv tcw cape
    ...                   ...
    ens_qtl_value:        0

In [3]:
pr_sc = pr_sc.sel(time=slice("1979-01-01", "2021-12-31"))
print("Días en el período:", pr_sc.time.size)

Días en el período: 15706


In [4]:
dry_day = (pr_sc < 1.0)

### Extraer dry spells (rachas True consecutivas) y filtrar ≥ 3 días

In [5]:
import numpy as np
import pandas as pd

def dry_spells_from_bool(time_index, dry_bool, min_len=3):
    dry = np.asarray(dry_bool, dtype=bool)

    x = np.r_[False, dry, False]
    dx = np.diff(x.astype(int))

    starts = np.where(dx == 1)[0]
    ends_excl = np.where(dx == -1)[0]
    lengths = ends_excl - starts

    mask = lengths >= min_len
    starts = starts[mask]
    ends_excl = ends_excl[mask]
    lengths = lengths[mask]

    t = np.asarray(time_index)
    start_dates = t[starts]
    end_dates = t[ends_excl - 1]

    df = pd.DataFrame({
        "start": pd.to_datetime(start_dates),
        "end": pd.to_datetime(end_dates),
        "duration_days": lengths.astype(int)
    }).sort_values("start").reset_index(drop=True)

    return df

df_spells = dry_spells_from_bool(
    time_index=pr_sc["time"].values,
    dry_bool=dry_day.values,
    min_len=3
)

df_spells.head(10)


,start,end,duration_days
0,1979-01-01,1979-01-10,10
1,1979-01-12,1979-02-07,27
2,1979-02-09,1979-04-03,54
3,1979-04-05,1979-04-26,22
4,1979-04-28,1979-05-01,4
5,1979-05-03,1979-05-15,13
6,1979-05-17,1979-06-21,36
7,1979-06-23,1979-07-24,32
8,1979-08-04,1979-08-23,20
9,1979-09-01,1979-09-06,6


Un día seco fue definido como aquel con precipitación diaria menor a 1 mm. Los dry spells se identificaron como secuencias de días secos consecutivos, considerando únicamente aquellos eventos con una duración mínima de tres días. La duración de cada dry spell se calculó como el número total de días consecutivos secos que componen cada evento.

### Calculamos la duración de todos los dry spells

In [6]:
duraciones = df_spells["duration_days"].to_numpy()
print("Cantidad de dry spells (>=3 días):", len(duraciones))
print("Primeras 20 duraciones:", duraciones[:20])


Cantidad de dry spells (>=3 días): 740
Primeras 20 duraciones: [10 27 54 22  4 13 36 32 20  6  3  8 38  4  3 16  6 57 10 47]


### Tomaremos el primer evento y veremos 

In [7]:
df_spells.iloc[0]


start            1979-01-01 00:00:00
end              1979-01-10 00:00:00
duration_days                     10
Name: 0, dtype: object

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- df_spells debe tener: start, end, duration_days ---
# Asegurar tipos:
df = df_spells.copy()
df["start"] = pd.to_datetime(df["start"])
df["end"] = pd.to_datetime(df["end"])
df["year"] = df["start"].dt.year  # asignación por año de inicio del evento

years = np.arange(1979, 2022)
g = df.groupby("year")["duration_days"]

annual = pd.DataFrame(index=years)
annual.index.name = "year"

# Promedio duración (años sin eventos -> NaN)
annual["mean_dur"] = g.mean().reindex(years)

# Más largo (años sin eventos -> NaN)
annual["max_dur"] = g.max().reindex(years)

# Percentil 99 (años con pocos eventos puede ser inestable;
annual["p99_dur"] = g.apply(lambda s: np.percentile(s, 99)).reindex(years)

annual["tL"] = g.apply(lambda s: np.percentile(s, 95)).reindex(years)

annual.head()


,mean_dur,max_dur,p99_dur,tL
year,,,,
1979,19.944444,57,56.49,54.45
1980,16.333333,128,109.37,45.80
1981,14.000000,63,56.44,30.20
1982,15.000000,70,66.60,53.00
1983,13.409091,73,67.33,44.75


In [9]:
annual.head()

,mean_dur,max_dur,p99_dur,tL
year,,,,
1979,19.944444,57,56.49,54.45
1980,16.333333,128,109.37,45.80
1981,14.000000,63,56.44,30.20
1982,15.000000,70,66.60,53.00
1983,13.409091,73,67.33,44.75


El parámetro tL fue definido como la duración típica máxima de los periodos secos, representando el umbral a partir del cual la probabilidad de observar dry spells de mayor duración decrece rápidamente. Operativamente, tL fue estimado como el percentil 95 de la distribución anual de duraciones de dry spells

- Le dejé P95 a tL porque es más estable, representa mejor el "límite típico"

In [10]:
rm = annual.rolling(window=5, min_periods=5).mean().shift(-4)
rm.head()


,mean_dur,max_dur,p99_dur,tL
year,,,,
1979,15.737374,78.2,71.246,45.64
1980,15.310390,94.8,84.948,47.75
1981,14.843723,86.2,77.914,44.79
1982,15.926076,90.4,82.722,52.03
1983,16.216076,101.4,90.830,48.57


In [11]:
rm.tail(10)


,mean_dur,max_dur,p99_dur,tL
year,,,,
2012,19.234888,87.4,83.066,65.73
2013,19.822190,107.6,100.476,71.98
2014,17.896036,89.4,83.632,60.56
2015,17.860742,90.4,83.384,55.32
2016,21.250742,95.2,88.706,62.73
2017,21.751092,94.8,88.818,64.89
2018,NaN,NaN,NaN,NaN
2019,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN


### Usamos información del futuro, no del pasado ni presente (podriamos hacer el caso del promedio, por ejemplo en 1981 tomaríamos 1979-1980-**1981**-1982-1983)
